In [1]:
from os import chdir
chdir('..')

In [4]:
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import matplotlib
import math
import seaborn as sns
import pandas as pd
import numpy as np
from polylidar import extractPlanesAndPolygons
from shapely.geometry import LineString, asPolygon
from shapely.affinity import scale, translate
from os import path

from concave_evaluation.helpers import fake_building, plot_line, BLUE, GRAY, plot_arrow, scale_axes, plot_poly, load_polygon, get_max_bounds_polys, measure_convexity_simple
%matplotlib inline
sns.set(font_scale=1)  # crazy big

In [22]:
file_path_dir = './test_fixtures/realsense'
file_name = 'raw_points.txt'
pc = np.loadtxt(path.join(file_path_dir, file_name))
# import open3d as o3d
# pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pc))
# o3d.visualization.draw_geometries([pcd])

In [23]:
mask = pc[:,2] < -1.0
pc_plane_only = pc[mask]
np.savetxt(path.join(file_path_dir, 'realsense_points.txt'), pc_plane_only[:, :2])